note: this notebook uses real data from a project work, source file is not shared here, not runable without it

After full extration of the salary reports from multiple projects and factories now we will upload the final dataset to the MySQL Database

In [3]:
# importing the libraries
import pandas as pd
import pymysql
import datetime
import numpy as np
from tqdm import tqdm

#### importing the final dataset to upload, the dataset's format is pickle

In [4]:
data = pd.read_pickle(r"E:\Google Drive Riyad IPA\HomeOffice\Merged Dataset Project\MDS_Uploaded_Datasets\CrossProject\CrossProject_Salary_MDS_20201223_Final")

In [5]:
#looking into the columns
data.columns

Index(['id', 'name', 'designation', 'doj', 'grade', 'attendance', 'absent',
       'total_holidays', 'total_leave', 'dayspaid', 'grosspay', 'basic',
       'house_allowance', 'medical_allowance', 'conveyance', 'food_allowance',
       'other_allowance', 'ot_hrs', 'ot_rate', 'ot_amount', 'arrear',
       'absent_deduction', 'provident_fund', 'tax_deduction', 'stamp', 'hr',
       'bf', 'advance', 'loan_deduction', 'total_deduction', 'netpay',
       'active', 'department', 'month', 'unit', 'section', 'desig_type_calc',
       'other_deduction', 'eot_hrs', 'eot_rate', 'eot_amount', 'doj_string',
       'analyst', 'project', 'fact_code', 'report_name', 'other_leave',
       'sick_leave', 'late_days', 'night_allowance',
       'grocery_advance_deduction', 'path', 'file', 'sheet',
       'attendance_bonus', 'line', 'floor', 'line_code', 'month_no', 'team',
       'gender', 'earned_leave', 'weekend_holidays', 'other_holidays',
       'factory_comments', 'night_day', 'extra_conveyance_days', 

In [6]:
#checking the length of the dataset
len(data)

137130

In [2]:
#factory 1502 is missing linecodes for the line H,let's fix it here
indices = data[data.fact_code == 1502][pd.isnull(data[data.fact_code == 1502]["line_code"])][data[data.fact_code == 1502][pd.isnull(data[data.fact_code == 1502]["line_code"])]["line"] == "H"].index

In [ ]:
data.loc[indices,"line_code"] = 1502020

In [ ]:
data[data.fact_code == 1502][pd.isnull(data[data.fact_code == 1502]["line_code"])]["line"].unique()

#### checking if all the columns in the dataset are actually present in the database

In [14]:
old_data = pd.read_csv(r"C:\Users\maahm\Downloads\salary.csv")

In [15]:
old_data.columns

Index(['slry_key', 'project', 'fact_code', 'month', 'month_no', 'department',
       'floor', 'section', 'unit', 'line', 'line_code', 'id', 'name',
       'designation', 'desig_type_calc', 'grade', 'doj', 'doj_string',
       'employee_category', 'gender', 'dob', 'dob_string', 'basic',
       'house_allowance', 'medical_allowance', 'conveyance', 'food_allowance',
       'grosspay', 'attendance', 'casual_leave', 'sick_leave', 'earned_leave',
       'paternity_leave', 'other_leave', 'weekend_holidays',
       'festival_holidays', 'maternity_leave', 'total_leave', 'other_holidays',
       'total_holidays', 'dayspaid', 'attendance_bonus', 'festival_allowance',
       'conduct_bonus', 'production_bonus', 'increment', 'other_allowance',
       'absent', 'absent_deduction', 'absent_halfday', 'advance',
       'conduct_deduction', 'grocery_advance_deduction', 'late_days',
       'late_join_deduction', 'provident_fund', 'without_pay_leave', 'stamp',
       'other_deduction', 'total_deduction', 

In [16]:
len(old_data)

5

In [17]:
data.columns

Index(['id', 'name', 'designation', 'doj', 'grade', 'attendance', 'absent',
       'total_holidays', 'total_leave', 'dayspaid', 'grosspay', 'basic',
       'house_allowance', 'medical_allowance', 'conveyance', 'food_allowance',
       'other_allowance', 'ot_hrs', 'ot_rate', 'ot_amount', 'arrear',
       'absent_deduction', 'provident_fund', 'tax_deduction', 'stamp', 'hr',
       'bf', 'advance', 'loan_deduction', 'total_deduction', 'netpay',
       'active', 'department', 'month', 'unit', 'section', 'desig_type_calc',
       'other_deduction', 'eot_hrs', 'eot_rate', 'eot_amount', 'doj_string',
       'analyst', 'project', 'fact_code', 'report_name', 'other_leave',
       'sick_leave', 'late_days', 'night_allowance',
       'grocery_advance_deduction', 'path', 'file', 'sheet',
       'attendance_bonus', 'line', 'floor', 'line_code', 'month_no', 'team',
       'gender', 'earned_leave', 'weekend_holidays', 'other_holidays',
       'factory_comments', 'night_day', 'extra_conveyance_days', 

In [18]:
[item for item in data.columns if item not in old_data.columns]

['hr',
 'bf',
 'active',
 'team',
 'mobile_allowance',
 'extra_conveyance_allowance',
 'lunch_out_days']

so these columns are missing, we have to fix some names in the source file and drop the rest

In [23]:
data.drop(['hr',
 'bf',
 'active',
 'team'],inplace=True,axis = 1)

In [24]:
len(old_data.columns), len(data.columns)

(88, 71)

In [25]:
data.head().style

,id,name,designation,doj,grade,attendance,absent,total_holidays,total_leave,dayspaid,grosspay,basic,house_allowance,medical_allowance,conveyance,food_allowance,other_allowance,ot_hrs,ot_rate,ot_amount,arrear,absent_deduction,provident_fund,tax_deduction,stamp,advance,loan_deduction,total_deduction,netpay,department,month,unit,section,desig_type_calc,other_deduction,eot_hrs,eot_rate,eot_amount,doj_string,analyst,project,fact_code,report_name,other_leave,sick_leave,late_days,night_allowance,grocery_advance_deduction,path,file,sheet,attendance_bonus,line,floor,line_code,month_no,gender,earned_leave,weekend_holidays,other_holidays,factory_comments,night_day,extra_conveyance_days,account_no,mobile_allowance,extra_conveyance_allowance,employee_category,maternity_leave,casual_leave,lunch_out_days,lunch_out_deduction
0,11,abdul awal,sr. supervisor,2006-06-06,1.000000,20.000000,0.000000,8.000000,3.000000,31.000000,25696.000000,15897.000000,7949.000000,600.000000,350.000000,900.000000,0.000000,0.000000,152.860000,0.000000,0.000000,0.000000,795.000000,0.000000,0.000000,527.000000,0.000000,1327.000000,24369.000000,administration,august_2020,nan,admin,nan,5.000000,0.000000,0.000000,0.000000,06_June_2006,NFH,CrossProject,1304,Salary Report,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2020_08,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,43,abu musa,sr. cook,2006-08-16,3.000000,23.000000,0.000000,8.000000,0.000000,31.000000,25303.000000,15635.000000,7818.000000,600.000000,350.000000,900.000000,0.000000,0.000000,150.340000,0.000000,0.000000,0.000000,782.000000,0.000000,0.000000,0.000000,0.000000,787.000000,24516.000000,administration,august_2020,nan,admin,nan,5.000000,0.000000,0.000000,0.000000,16_August_2006,NFH,CrossProject,1304,Salary Report,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2020_08,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,3064,md khalilur rahman,gardener,2007-08-11,4.000000,15.000000,8.000000,8.000000,0.000000,23.000000,24100.000000,14833.000000,7417.000000,600.000000,350.000000,900.000000,0.000000,0.000000,142.630000,0.000000,0.000000,3955.000000,742.000000,0.000000,0.000000,0.000000,0.000000,747.000000,19398.000000,administration,august_2020,nan,admin,nan,5.000000,0.000000,0.000000,0.000000,11_August_2007,NFH,CrossProject,1304,Salary Report,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2020_08,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,3468,md joynal abedin,office assistant,2007-11-01,3.000000,20.000000,0.000000,8.000000,3.000000,31.000000,25402.000000,15701.000000,7851.000000,600.000000,350.000000,900.000000,0.000000,0.000000,150.970000,0.000000,0.000000,0.000000,785.000000,0.000000,0.000000,0.000000,0.000000,790.000000,24612.000000,administration,august_2020,nan,admin,nan,5.000000,0.000000,0.000000,0.000000,01_November_2007,NFH,CrossProject,1304,Salary Report,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2020_08,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,5233,mohibulla titu,office attendant,2008-07-01,4.000000,23.000000,0.000000,8.000000,0.000000,31.000000,17799.000000,10633.000000,5316.000000,600.000000,350.000000,900.000000,0.000000,34.000000,102.240000,3476.000000,0.000000,0.000000,532.000000,0.000000,0.000000,0.000000,15000.000000,15537.000000,5738.000000,administration,august_2020,nan,admin,nan,5.000000,30.000000,102.240000,3067.000000,01_July_2008,NFH,CrossProject,1304,Salary Report,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2020_08,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [26]:
data.reset_index(drop=True,inplace=True)

In [29]:
#dropping duplicates if there are any
data.drop_duplicates(inplace=True)
data.reset_index(drop=True,inplace=True)

In [30]:
len(data)

137130

In [31]:
any(data.columns.value_counts().values == 2)

False

#### check dates' format before uploading

In [ ]:
data.date.unique()[:5]

In [ ]:
data.doj.unique()

In [ ]:
data['doj_string'] = pd.Series([e.strftime("%d_%B_%Y") if pd.notnull(e) else e for e in data.doj], index=data.index)

In [ ]:
data.del_date.unique()

In [ ]:
data["del_date"] = pd.Series([datetime.datetime.strptime(item,"%Y-%m-%d").date() if pd.notnull(item) else np.nan for item in data.del_date],index=data.index)

In [ ]:
data['del_date_string'] = pd.Series([e.strftime("%d_%B_%Y") if pd.notnull(e) else e for e in data.del_date], index=data.index)

#### null to None - MySQL takes nulls as black string, hence changing the type

In [33]:
data = data.where((pd.notnull(data)), None)

In [34]:
data.head().style

,id,name,designation,doj,grade,attendance,absent,total_holidays,total_leave,dayspaid,grosspay,basic,house_allowance,medical_allowance,conveyance,food_allowance,other_allowance,ot_hrs,ot_rate,ot_amount,arrear,absent_deduction,provident_fund,tax_deduction,stamp,advance,loan_deduction,total_deduction,netpay,department,month,unit,section,desig_type_calc,other_deduction,eot_hrs,eot_rate,eot_amount,doj_string,analyst,project,fact_code,report_name,other_leave,sick_leave,late_days,night_allowance,grocery_advance_deduction,path,file,sheet,attendance_bonus,line,floor,line_code,month_no,gender,earned_leave,weekend_holidays,other_holidays,factory_comments,night_day,extra_conveyance_days,account_no,mobile_allowance,extra_conveyance_allowance,employee_category,maternity_leave,casual_leave,lunch_out_days,lunch_out_deduction
0,11,abdul awal,sr. supervisor,2006-06-06,1.000000,20.000000,0.000000,8,3.000000,31.000000,25696.000000,15897.000000,7949.000000,600.000000,350.000000,900.000000,0.000000,0.000000,152.860000,0.000000,0.000000,0.000000,795.000000,0.000000,0.000000,527.000000,0.000000,1327.000000,24369.000000,administration,august_2020,None,admin,None,5.000000,0.000000,0.000000,0.000000,06_June_2006,NFH,CrossProject,1304,Salary Report,None,None,None,None,None,None,None,None,None,None,None,None,2020_08,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,43,abu musa,sr. cook,2006-08-16,3.000000,23.000000,0.000000,8,0.000000,31.000000,25303.000000,15635.000000,7818.000000,600.000000,350.000000,900.000000,0.000000,0.000000,150.340000,0.000000,0.000000,0.000000,782.000000,0.000000,0.000000,0.000000,0.000000,787.000000,24516.000000,administration,august_2020,None,admin,None,5.000000,0.000000,0.000000,0.000000,16_August_2006,NFH,CrossProject,1304,Salary Report,None,None,None,None,None,None,None,None,None,None,None,None,2020_08,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,3064,md khalilur rahman,gardener,2007-08-11,4.000000,15.000000,8.000000,8,0.000000,23.000000,24100.000000,14833.000000,7417.000000,600.000000,350.000000,900.000000,0.000000,0.000000,142.630000,0.000000,0.000000,3955.000000,742.000000,0.000000,0.000000,0.000000,0.000000,747.000000,19398.000000,administration,august_2020,None,admin,None,5.000000,0.000000,0.000000,0.000000,11_August_2007,NFH,CrossProject,1304,Salary Report,None,None,None,None,None,None,None,None,None,None,None,None,2020_08,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,3468,md joynal abedin,office assistant,2007-11-01,3.000000,20.000000,0.000000,8,3.000000,31.000000,25402.000000,15701.000000,7851.000000,600.000000,350.000000,900.000000,0.000000,0.000000,150.970000,0.000000,0.000000,0.000000,785.000000,0.000000,0.000000,0.000000,0.000000,790.000000,24612.000000,administration,august_2020,None,admin,None,5.000000,0.000000,0.000000,0.000000,01_November_2007,NFH,CrossProject,1304,Salary Report,None,None,None,None,None,None,None,None,None,None,None,None,2020_08,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,5233,mohibulla titu,office attendant,2008-07-01,4.000000,23.000000,0.000000,8,0.000000,31.000000,17799.000000,10633.000000,5316.000000,600.000000,350.000000,900.000000,0.000000,34.000000,102.240000,3476.000000,0.000000,0.000000,532.000000,0.000000,0.000000,0.000000,15000.000000,15537.000000,5738.000000,administration,august_2020,None,admin,None,5.000000,30.000000,102.240000,3067.000000,01_July_2008,NFH,CrossProject,1304,Salary Report,None,None,None,None,None,None,None,None,None,None,None,None,2020_08,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [35]:
len(data)

137130

In [38]:
# Connect to the database
connection = pymysql.connect(host='earth.mydchub.com',
                             user='rmgppcom_mrr',
                             password='IP@MRR#1T',
                             db='rmgppcom_db')

In [39]:
cursor = connection.cursor()

In [40]:
# creating column list for insertion
cols = "`,`".join([str(i) for i in data.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in tqdm(data.iterrows()):
    sql = "INSERT INTO `salary` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

137130it [15:12, 150.26it/s]


In [41]:
len(data)

137130

In [42]:
connection.close()

In [43]:
print ("DONE")

DONE
